In [ ]:
import pandas as pd
import gemmi
import matplotlib.pyplot as plt

output_labels_csv_name = '/dls/labxchem/data/2018/lb18145-80/processing/analysis/eugene/panddaproject/score_examples/NSP14-x1069/output_labels.csv'

df = pd.read_csv(output_labels_csv_name)

def read_chain_name(input_model: str,
                    input_chain_idx: int):
    structure = gemmi.read_structure(input_model)[0]
    chain_name = structure[input_chain_idx].name
    return chain_name

def list_residues_and_pred_labels(output_labels_dataframe: pd.DataFrame):
    input_model_list = output_labels_dataframe['input_model'].tolist()
    input_chain_idx_list = output_labels_dataframe['input_chain_idx'].tolist()
    chain_names = list(map(read_chain_name, 
                            input_model_list, 
                            input_chain_idx_list))
    residue_names = output_labels_dataframe['residue_name'].tolist()
    chain_residue_names = list(map('-'.join, zip(chain_names, residue_names)))
    pred_labels = output_labels_dataframe['pred_label'].tolist()

    return residue_names, pred_labels

def plot_residue_vs_pred_label(residue_names: list, pred_labels: list):
    plt.figure(figsize=(20, 10))
    plt.bar(residue_names, pred_labels)
    plt.xticks(rotation=90)
    plt.show()

residue_names, pred_labels = list_residues_and_pred_labels(df)
plot_residue_vs_pred_label(residue_names, pred_labels)

print(residue_names)

